# Evaluate Model Performance
Evalute model perfomance using PSNR, SSIM and SAM

## Evaluate on real data

In [ ]:
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import pandas as pd
import numpy as np
import torch
from torchmetrics.image import SpectralAngleMapper

def compute_metrics(root_dir, model):
    hr_path = root_dir + "c_hr/"
    sr_path = root_dir + "models/" + model + "/sr/"

    scores = []
    img_id_list = os.listdir(hr_path)
    for img_id in img_id_list:
        print("Evaluating image: ", img_id)
        
        # Load images
        img_hr = np.load(os.path.join(hr_path, img_id))
        img_sr = np.load(os.path.join(sr_path, img_id))

        # Load as tensors
        img_hr_tensor = torch.tensor(img_hr, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
        img_sr_tensor = torch.tensor(img_sr, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)

        psnr_value = psnr(img_hr, img_sr, data_range=img_hr.max() - img_hr.min())
        ssim_value = ssim(img_hr, img_sr, data_range=img_hr.max() - img_hr.min(), channel_axis=2)
        sam_metric = SpectralAngleMapper() # Read what this does
        sam_value = sam_metric(img_hr_tensor, img_sr_tensor).item()
        scores.append({'Image_ID': img_id, 'PSNR': psnr_value, 'SSIM': ssim_value, 'SAM': sam_value})

    return pd.DataFrame(scores)



# Choose model to evaluate
model = "HSI_x2_real"
data_dir = "results/"

# Scores for SR and bicubic in a dataframe
df_sr = compute_metrics(root_dir=data_dir, model=model)

# Save the mean scores to txt
with open(f"{data_dir}/models/{model}/{model}_means.txt", "w") as f:
    f.write("PSNR: " + str(round(df_sr['PSNR'].mean(), 4)) + "\n")
    f.write("SSIM: " + str(round(df_sr['SSIM'].mean(), 4)) + "\n")
    f.write("SAM: " + str(round(df_sr['SAM'].mean(), 4)) + "\n")

# Print mean of SR and BI
print("Mean PSNR SR: ", round(df_sr['PSNR'].mean(), 4))
print("Mean SSIM SR: ", round(df_sr['SSIM'].mean(), 4))
print("Mean SAM SR: ", round(df_sr['SAM'].mean(), 4))

# Save the dataframe to a csv file
df_sr.to_csv(f"{data_dir}/models/{model}/{model}_scores.csv", index=False)

## Evaluate on synthetic data

In [ ]:
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import pandas as pd
import numpy as np
import torch
from torchmetrics.image import SpectralAngleMapper

def compute_metrics(root_dir, model):
    hr_path = root_dir + "c_hr/"
    sr_path = root_dir + "models/" + model + "/sr_synth/"

    scores = []
    img_id_list = os.listdir(hr_path)
    for img_id in img_id_list:
        print("Evaluating image: ", img_id)
        
        # Load images
        img_hr = np.load(os.path.join(hr_path, img_id))
        img_sr = np.load(os.path.join(sr_path, img_id))

        # Load as tensors
        img_hr_tensor = torch.tensor(img_hr, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
        img_sr_tensor = torch.tensor(img_sr, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)

        psnr_value = psnr(img_hr, img_sr, data_range=img_hr.max() - img_hr.min())
        ssim_value = ssim(img_hr, img_sr, data_range=img_hr.max() - img_hr.min(), channel_axis=2)
        sam_metric = SpectralAngleMapper() # Read what this does
        sam_value = sam_metric(img_hr_tensor, img_sr_tensor).item()
        scores.append({'Image_ID': img_id, 'PSNR': psnr_value, 'SSIM': ssim_value, 'SAM': sam_value})

    return pd.DataFrame(scores)



# Choose model to evaluate
model = "HSI_x2_real"
data_dir = "results/"

# Scores for SR and bicubic in a dataframe
df_sr = compute_metrics(root_dir=data_dir, model=model)

# Save the mean scores to txt
with open(f"{data_dir}/models/{model}/{model}_synth_means.txt", "w") as f:
    f.write("PSNR: " + str(round(df_sr['PSNR'].mean(), 4)) + "\n")
    f.write("SSIM: " + str(round(df_sr['SSIM'].mean(), 4)) + "\n")
    f.write("SAM: " + str(round(df_sr['SAM'].mean(), 4)) + "\n")

# Print mean of SR and BI
print("Mean PSNR SR: ", round(df_sr['PSNR'].mean(), 4))
print("Mean SSIM SR: ", round(df_sr['SSIM'].mean(), 4))
print("Mean SAM SR: ", round(df_sr['SAM'].mean(), 4))

# Save the dataframe to a csv file
df_sr.to_csv(f"{data_dir}/models/{model}/{model}_synth_scores.csv", index=False)